In [ ]:
import torch as tn
import torchtt as tntt
import TTCME
import matplotlib.pyplot as plt 
import scipy.integrate
import numpy as np
import datetime

tn.set_default_tensor_type(tn.DoubleTensor)

r1 = TTCME.ChemicalReaction(['mRNA','protein'],'mRNA->mRNA+protein', 0.015)
r2 = TTCME.ChemicalReaction(['mRNA','protein'],'mRNA->', 0.002)
r3 = TTCME.ChemicalReaction(['mRNA','protein'],'->mRNA', 0.1)
r4 = TTCME.ChemicalReaction(['mRNA','protein'],'protein->', 0.01)

rs = TTCME.ReactionSystem(['mRNA','protein'],[r1, r2, r3, r4])


N = [80,120] 

Att = rs.generatorTT(N)

integrator = TTCME.TimeIntegrator.TTInt(Att, N_max=16, dt_max = 100, method='legendre')

p0 = TTCME.pdf.SingularPMF(N,[2,4])
p = p0.copy()

dt = 128 

time = 0
for i in range(8):
    tme = datetime.datetime.now()
    p.dofs = integrator.solve(p.dofs, dt, intervals = 1)
    tme = datetime.datetime.now() - tme
    time += dt
    print('t = %g, solver time %g s'%(time,tme.total_seconds()))



In [ ]:
Asp = rs.generator_sparse(N)

res = scipy.integrate.solve_ivp(lambda t,y: Asp.dot(y), [0,time],p0.dofs.numpy().flatten(),max_step=time/10000)
Pt = res.y.reshape(N+[-1])
p_ref = Pt[:,:,-1]

In [ ]:
plt.figure()
plt.imshow(p.dofs.full(), origin='lower',cmap='gray_r')
plt.colorbar()
plt.xlabel(r'$x_1$')
plt.ylabel(r'$x_2$')

plt.figure()
plt.imshow(p_ref, origin='lower',cmap='gray_r')
plt.colorbar()
plt.xlabel(r'$x_1$')
plt.ylabel(r'$x_2$')

plt.figure()
plt.imshow(np.abs(p.dofs.numpy()-p_ref), origin='lower',cmap='gray_r')
plt.colorbar()
plt.xlabel(r'$x_1$')
plt.ylabel(r'$x_2$')